# Project: Recognize faces to put on characters

## Face Detection vs Face Recognition

Package Install(패키지 설치)
> pip install mediapipe

Sample Video : https://www.pexels.com/video/3256542/  
Size : Full HD (1920 x 1080)  
File name : face_video.mp4

Character image : https://www.freepik.com/free-vector/cute-animal-masks-video-chat-application-effect-filters-set_6380101.htm  
File name : right_eye.png (100 x 100), left_eye.png (100 x 100), nose.png (300 x 100)

In [11]:
!pip install -q sounddevice
!pip install -q -i https://test.pypi.org/simple/ mediapipe==0.9.3.2        

ERROR: Could not find a version that satisfies the requirement mediapipe==0.9.3.2 (from versions: 0.8.4.1, 0.8.4.2, 0.8.4.8, 0.8.6, 0.8.6.1, 0.8.7, 0.8.7.1, 0.8.8, 0.8.9, 0.8.10, 0.8.11, 0.9.2.2)
ERROR: No matching distribution found for mediapipe==0.9.3.2


## Face Detection

In [12]:
import cv2
import mediapipe as mp

# Define variables to find faces and mark the ones you find
mp_face_detection = mp.solutions.face_detection # Use the face_detection module for face detection
mp_drawing = mp.solutions.drawing_utils # Using the drawing_utils module to draw facial features

# Open a video file
cap = cv2.VideoCapture('face_video.mp4')

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR을 RGB로 변경
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #BGR을 RGB로 변경
        
        if results.detections:
            # 6 features: right eye, left eye, tip of nose, center of mouth, right ear, left ear (earlobe point, migration)
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)          
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

## Recognize faces to put on characters

In [8]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image): # Target image (3 channels), x, y coordinates, width, height, image to overwrite (4 channels)
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 to 255 -> A value between 0 and 1 when divided by 255 (1: opaque, 0: fully transparent)
    # (255, 255)  ->  (1, 1)
    # (255, 0)        (1, 0)
    
    # 1 - mask_image ?
    # (0, 0)
    # (0, 1)
    
    for c in range(0, 3): # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

# Define variables to find faces and mark the ones you find
mp_face_detection = mp.solutions.face_detection # Use the face_detection module for face detection
mp_drawing = mp.solutions.drawing_utils # Using the drawing_utils module to draw facial features

# Open a video file
cap = cv2.VideoCapture('face_video.mp4')

# Import images (right eye, left eye, nose)
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100 (Horizontal, Vertical)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #Change BGR to RGB
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #Change BGR to RGB
        
        if results.detections:
            # 6 features: right eye, left eye, tip of nose, center of mouth, right ear, left ear (earlobe point, tragus: just in front of the earlobe)
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # Get a specific location
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # Right eye
                left_eye = keypoints[1] # Left eye
                nose_tip = keypoints[2] # Nose Tip
                
                h, w, _ = image.shape # height, width, channel : Get vertical and horizontal dimensions from an image
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # Real-world coordinates (x, y) within an image
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # Circle both eyes
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 파란색
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색                
                # Circle the nose
                # cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA) # 노란색
                
                # Draw an image for each feature
                # image_right_eye.shape
                # image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                # image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                # image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = image_nose
                
                # image, x, y, w, h, overlay_image
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 150, 50, image_nose)
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()